# Using Random Forest classifier for the data on reviews

In [20]:
import pandas as pd
from sklearn.metrics import matthews_corrcoef as mcc

In [2]:
dfLinesAll = pd.read_csv('./train_lines.csv', sep='$')

In [3]:
print(dfLinesAll.head())
dfLinesAll = dfLinesAll.rename(columns = {'fit': 'fit_feature'})

                                                line  serializer->doReal  \
0              serializer->doReal(&iparams->cross...                   1   
1        #include "gromacs/mdtypes/iforceprovider.h"                   0   
2  /* Target mantissa accuracy forSIMD double pre...                   0   
3                     cdh.startFrame(frnr, fr.time)_                   0   
4                             sfree(mtop->molblock)_                   0   

   &iparams->cross_bb  krr  #include  gromacs/mdtypes/iforceprovider  /*  \
0                   1    1         0                               0   0   
1                   0    0         1                               1   0   
2                   0    0         0                               0   1   
3                   0    0         0                               0   0   
4                   0    0         0                               0   0   

   Target  cdh  startFrame     ...       minres_  HAVE_RDTSCP  \
0       0    0       

In [4]:
dfLinesAll.groupby('class_value')['line'].count()

class_value
0    21153
1    16482
Name: line, dtype: int64

In [5]:
X = dfLinesAll.drop(['class_value', 'line'], axis=1)
y = dfLinesAll['class_value']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=128, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test)

accuracy_score(y_test, y_pred_rf)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


0.6748792270531401

In [21]:
from sklearn.metrics import f1_score, classification_report
print(f1_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))
print(mcc(y_test, y_pred))

0.48257303946693997
             precision    recall  f1-score   support

          0       0.64      0.94      0.76      6932
          1       0.81      0.34      0.48      5488

avg / total       0.72      0.67      0.64     12420



NameError: name 'y_pred' is not defined

# Ada boost classifier

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
ada_clf = AdaBoostClassifier(
 DecisionTreeClassifier(max_depth=100), n_estimators=200,
 algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=100,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=0.5, n_estimators=200, random_state=None)

# Model validation

In [12]:
from sklearn.metrics import f1_score
y_pred_ab = ada_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred_ab)
f1 = f1_score(y_test, y_pred_ab)
mcc1 = mcc(y_test, y_pred)

print(f'Accuracy: {acc:.3f}, F1: {f1:.3f}, mcc: {mcc1:.3f}')
print(classification_report(y_test, y_pred_ab))

Accuracy: 0.749, F1: 0.677
             precision    recall  f1-score   support

          0       0.73      0.87      0.79      6932
          1       0.78      0.60      0.68      5488

avg / total       0.75      0.75      0.74     12420



In [ ]:
from sklearn.model_selection import cross_val_score
scoring_to_get = ['accuracy', 'precision', 'f1', 'recall']
cross_val_score(ada_clf, X_train, y_train, cv=10, scoring=scoring_to_get)

In [ ]:
from sklearn import metrics
#metrics.SCORERS.keys()

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
num_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
        ('rf', RandomForestClassifier()),
    ])

num_pipeline.fit(X_train, y_train)

y_pred_ab = num_pipeline.predict(X_test)

acc = accuracy_score(y_test, y_pred_ab)
f1 = f1_score(y_test, y_pred_ab)

mcc1 = mcc(y_test, y_pred)

print(f'Accuracy: {acc:.3f}, F1: {f1:.3f}, mcc: {mcc1:.3f}')
print(classification_report(y_test, y_pred_ab))
print(confusion_matrix(y_test, y_pred_ab))

Accuracy: 0.711, F1: 0.618
             precision    recall  f1-score   support

          0       0.70      0.86      0.77      6932
          1       0.74      0.53      0.62      5488

avg / total       0.72      0.71      0.70     12420

[[5933  999]
 [2589 2899]]


In [ ]:
from sklearn.model_selection import cross_val_score
#cross_val_score(num_pipeline, X_train, y_train, cv=10, scoring=scoring_to_get)
#cross_val_score(num_pipeline, X_train, y_train, cv=10, scoring='accuracy')
#cross_val_score(num_pipeline, X_train, y_train, cv=10, scoring='precision')
#cross_val_score(num_pipeline, X_train, y_train, cv=10, scoring='recall')

In [18]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


ab_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
        ('ab', AdaBoostClassifier(DecisionTreeClassifier(max_depth=100), n_estimators=200, algorithm="SAMME.R", learning_rate=0.2)),
    ])

ab_pipeline.fit(X_train, y_train)

y_pred_ab = ab_pipeline.predict(X_test)

acc = accuracy_score(y_test, y_pred_ab)
f1 = f1_score(y_test, y_pred_ab)

mcc1 = mcc(y_test, y_pred)

print(f'Accuracy: {acc:.3f}, F1: {f1:.3f}, mcc: {mcc1:.3f}')
print(classification_report(y_test, y_pred_ab))
print(confusion_matrix(y_test, y_pred_ab))
#cross_val_score(ab_pipeline, X, y, cv=10, scoring=scoring_to_get)

Accuracy: 0.675, F1: 0.553
             precision    recall  f1-score   support

          0       0.66      0.85      0.75      6932
          1       0.71      0.45      0.55      5488

avg / total       0.68      0.68      0.66     12420

[[5894 1038]
 [2995 2493]]


In [18]:
#cross_val_score(ab_pipeline, X, y, cv=10, scoring='precision')

array([0.70985915, 0.75119617, 0.70379538, 0.74084507, 0.74901575,
       0.65107459, 0.71698113, 0.7248062 , 0.74209012, 0.75557207])

In [19]:
#cross_val_score(ab_pipeline, X, y, cv=10, scoring='recacoll')

array([0.60885385, 0.63432383, 0.63046117, 0.62196602, 0.46480583,
       0.6086165 , 0.61286408, 0.4526699 , 0.45995146, 0.49635922])

In [20]:
#cross_val_score(ab_pipeline, X, y, cv=10, scoring='accuracy')

array([0.70756972, 0.70464807, 0.69341126, 0.73638055, 0.70289663,
       0.6986447 , 0.70395961, 0.69731597, 0.69545575, 0.7214988 ])

In [19]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score


ab_pipeline = Pipeline([
        ('std_scaler', StandardScaler()),
        ('ab', AdaBoostClassifier(DecisionTreeClassifier(max_depth=20), n_estimators=600, algorithm="SAMME.R", learning_rate=0.2)),
    ])

ab_pipeline.fit(X_train, y_train)

y_pred_ab = ab_pipeline.predict(X_tmatthewsest)

acc = accuracy_score(y_test, y_pred_ab)
f1 = f1_score(y_test, y_pred_ab)

mcc1 = mcc(y_test, y_pred)

print(f'Accuracy: {acc:.3f}, F1: {f1:.3f}, mcc: {mcc1:.3f}')
print(classification_report(y_test, y_pred_ab))
print(confusion_matrix(y_test, y_pred_ab))

#cross_val_score(ab_pipeline, X, y, cv=10, scoring=scoring_to_get)

Accuracy: 0.731, F1: 0.666
             precision    recall  f1-score   support

          0       0.73      0.83      0.77      6932
          1       0.74      0.61      0.67      5488

avg / total       0.73      0.73      0.73     12420

[[5739 1193]
 [2153 3335]]


In [22]:
#cross_val_score(ab_pipeline, X, y, cv=10, scoring='precision')

array([0.75134305, 0.76412962, 0.7896365 , 0.74066244, 0.76654804,
       0.76223242, 0.76309795, 0.74690007, 0.76560122, 0.74428571])

In [23]:
#cross_val_score(ab_pipeline, X, y, cv=10, scoring='recall')

array([0.60157671, 0.62765312, 0.59648058, 0.61104369, 0.63288835,
       0.62621359, 0.61529126, 0.59101942, 0.61286408, 0.64866505])

In [24]:
#cross_val_score(ab_pipeline, X, y, cv=10, scoring='accuracy')

array([0.73678619, 0.7373174 , 0.73963868, 0.74355567, 0.74249269,
       0.73584906, 0.73372309, 0.74063247, 0.73425458, 0.73505182])